# 02 - EDA y Feature Engineering

**Proyecto:** Seguridad Inteligente con Arduino y Visión Artificial

Este notebook realiza:
1. Análisis Exploratorio de Datos (EDA)
2. Ingeniería de Características (Feature Engineering)
3. Generación de visualizaciones y reportes

---

## 1. Configuración Inicial

In [ ]:
# Importar librerías necesarias
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Clases FeatureEngineering y EDAAnalyzer se definen en la siguiente celda
# from features import FeatureEngineering
# from eda import EDAAnalyzer

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

# Opciones de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuración de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✓ Librerías importadas correctamente")

In [ ]:
# Clases internas para trabajar sin carpeta src
import yaml
from typing import Dict

class FeatureEngineering:
    def __init__(self, config_path: str = "configs/config.yaml"):
        with open(config_path, 'r') as f:
            self.config = yaml.safe_load(f)
        self.processed_path = Path(self.config['data']['processed_path'])
        self.time_window = self.config['features']['time_window']

    def load_data(self, filename: str = None) -> pd.DataFrame:
        if filename is None:
            filename = self.config['data']['output_file']
        file_path = self.processed_path / filename
        return pd.read_csv(file_path, parse_dates=['date'])

    def create_temporal_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        df['period_of_day'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 24], labels=['Madrugada', 'Mañana', 'Tarde', 'Noche'], include_lowest=True)
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        return df

    def create_event_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy().sort_values('date')
        df['events_in_window'] = df.rolling(window=f"{self.time_window}min", on='date').size()
        df['time_since_last'] = df['date'].diff().dt.total_seconds() / 60
        df['time_to_next'] = -df['date'].diff(-1).dt.total_seconds() / 60
        df['events_per_hour'] = df.groupby([df['date'].dt.date, df['hour']])['id'].transform('count')
        return df

    def create_text_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df['text_length'] = df['text'].fillna('').str.len()
        df['word_count'] = df['text'].fillna('').str.split().str.len()
        keywords = ['alarma', 'movimiento', 'puerta', 'sensor', 'alerta']
        for keyword in keywords:
            df[f'has_{keyword}'] = df['text'].fillna('').str.lower().str.contains(keyword).astype(int)
        return df

    def create_aggregated_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        daily_counts = df.groupby(df['date'].dt.date).size()
        df['events_same_day'] = df['date'].dt.date.map(daily_counts)
        hourly_counts = df.groupby('hour').size()
        df['avg_events_this_hour'] = df['hour'].map(hourly_counts)
        photo_counts = df[df['has_photo']].groupby(df['date'].dt.date).size()
        df['photos_same_day'] = df['date'].dt.date.map(photo_counts).fillna(0)
        return df

    def save_features(self, df: pd.DataFrame, filename: str = "events_features.csv"):
        output_path = self.processed_path / filename
        output_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(output_path, index=False, encoding='utf-8')

class EDAAnalyzer:
    def __init__(self, config_path: str = "configs/config.yaml"):
        with open(config_path, 'r') as f:
            self.config = yaml.safe_load(f)
        self.processed_path = Path(self.config['data']['processed_path'])
        self.reports_path = Path(self.config['reports']['output_path'])
        self.figures_path = self.reports_path / "figures"
        self.figures_path.mkdir(parents=True, exist_ok=True)

    def generate_summary_stats(self, df: pd.DataFrame) -> Dict:
        return {
            'total_events': len(df),
            'date_range': {
                'start': df['date'].min(),
                'end': df['date'].max(),
                'days': (df['date'].max() - df['date'].min()).days
            },
            'events_with_photos': df['has_photo'].sum() if 'has_photo' in df.columns else 0,
            'unique_senders': df['from'].nunique() if 'from' in df.columns else 0,
            'avg_events_per_day': len(df) / ((df['date'].max() - df['date'].min()).days + 1),
            'hourly_distribution': df['hour'].value_counts().to_dict() if 'hour' in df.columns else {},
            'daily_distribution': df['day_name'].value_counts().to_dict() if 'day_name' in df.columns else {}
        }

    def plot_time_series(self, df: pd.DataFrame):
        plt.figure(figsize=(14, 6))
        daily_events = df.groupby(df['date'].dt.date).size()
        plt.plot(daily_events.index, daily_events.values, marker='o', linewidth=2)
        plt.title('Eventos de Seguridad a lo Largo del Tiempo', fontsize=14, fontweight='bold')
        plt.xlabel('Fecha')
        plt.ylabel('Número de Eventos')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        output_path = self.figures_path / "time_series.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_hourly_distribution(self, df: pd.DataFrame):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        hourly = df['hour'].value_counts().sort_index()
        ax1.bar(hourly.index, hourly.values, color='steelblue', alpha=0.7)
        ax1.set_title('Distribución de Eventos por Hora del Día', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Hora del Día')
        ax1.set_ylabel('Número de Eventos')
        ax1.grid(axis='y', alpha=0.3)
        if 'day_name' in df.columns:
            import seaborn as sns
            pivot = df.groupby(['hour', 'day_name']).size().unstack(fill_value=0)
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            pivot = pivot.reindex(columns=day_order, fill_value=0)
            sns.heatmap(pivot, cmap='YlOrRd', annot=True, fmt='d', ax=ax2, cbar_kws={'label': 'Eventos'})
            ax2.set_title('Heatmap: Eventos por Hora y Día', fontsize=12, fontweight='bold')
            ax2.set_xlabel('Día de la Semana')
            ax2.set_ylabel('Hora del Día')
        plt.tight_layout()
        output_path = self.figures_path / "hourly_distribution.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_daily_distribution(self, df: pd.DataFrame):
        plt.figure(figsize=(10, 6))
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        day_counts = df['day_name'].value_counts().reindex(day_order)
        colors = ['#ff6b6b' if day in ['Saturday', 'Sunday'] else '#4ecdc4' for day in day_order]
        plt.bar(range(len(day_counts)), day_counts.values, color=colors, alpha=0.7)
        plt.xticks(range(len(day_counts)), day_counts.index, rotation=45)
        plt.title('Distribución de Eventos por Día de la Semana', fontsize=14, fontweight='bold')
        plt.xlabel('Día de la Semana')
        plt.ylabel('Número de Eventos')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        output_path = self.figures_path / "daily_distribution.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_photo_analysis(self, df: pd.DataFrame):
        if 'has_photo' not in df.columns:
            return
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        photo_counts = df['has_photo'].value_counts()
        colors = ['#ff7f0e', '#2ca02c']
        ax1.pie(photo_counts.values, labels=['Sin Foto', 'Con Foto'], autopct='%1.1f%%', startangle=90, colors=colors)
        ax1.set_title('Proporción de Eventos con Fotografía', fontsize=12, fontweight='bold')
        if 'hour' in df.columns:
            photo_by_hour = df[df['has_photo']].groupby('hour').size()
            ax2.bar(photo_by_hour.index, photo_by_hour.values, color='coral', alpha=0.7)
            ax2.set_title('Eventos con Foto por Hora del Día', fontsize=12, fontweight='bold')
            ax2.set_xlabel('Hora del Día')
            ax2.set_ylabel('Número de Eventos')
            ax2.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        output_path = self.figures_path / "photo_analysis.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

    def generate_report(self, df: pd.DataFrame, stats: Dict):
        from datetime import datetime
        report_path = self.reports_path / "informe_EDA.md"
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("# Informe de Análisis Exploratorio de Datos (EDA)\n\n")
            f.write(f"**Fecha de generación:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write("---\n\n")
            f.write("## 1. Resumen Ejecutivo\n\n")
            f.write(f"- **Total de eventos registrados:** {stats['total_events']:,}\n")
            f.write(f"- **Período analizado:** {stats['date_range']['start']} a {stats['date_range']['end']}\n")
            f.write(f"- **Duración:** {stats['date_range']['days']} días\n")
            f.write(f"- **Eventos con fotografía:** {stats['events_with_photos']:,}\n")
            f.write(f"- **Promedio de eventos por día:** {stats['avg_events_per_day']:.2f}\n\n")
            f.write("---\n\n")
            f.write("## 2. Análisis Temporal\n\n")
            f.write("![Serie Temporal](figures/time_series.png)\n\n")
            f.write("![Distribución Horaria](figures/hourly_distribution.png)\n\n")
            f.write("![Distribución Diaria](figures/daily_distribution.png)\n\n")
            f.write("---\n\n")
            f.write("## 3. Análisis de Fotografías\n\n")
            f.write("![Análisis de Fotos](figures/photo_analysis.png)\n\n")
            f.write("---\n\n")
            f.write("## 4. Conclusiones y Recomendaciones\n\n")
            f.write("1. Patrón temporal de eventos identificado\n")
            f.write("2. Distribución de eventos a lo largo del día y la semana\n")
            f.write("3. Proporción de eventos con evidencia fotográfica\n")
        

## 2. Cargar Datos Limpios

In [ ]:
# Cargar datos procesados del notebook anterior
data_path = Path('../data/processed/events_clean.csv')

if data_path.exists():
    df = pd.read_csv(data_path, parse_dates=['date'])
    print(f"✓ Datos cargados: {len(df)} registros")
    print(f"  Columnas: {list(df.columns)}")
    display(df.head())
else:
    print("⚠ Error: No se encontró el archivo de datos limpios")
    print("   Por favor, ejecuta primero el notebook 01_ingesta_y_limpieza.ipynb")
    df = None

## 3. Feature Engineering

Generaremos características adicionales para enriquecer los datos.

In [ ]:
# Inicializar el generador de features
if df is not None:
    feature_eng = FeatureEngineering(config_path='../configs/config.yaml')
    print("✓ Feature Engineering inicializado")

### 3.1 Características Temporales

In [ ]:
# Generar características temporales
if df is not None:
    df_features = feature_eng.create_temporal_features(df)
    
    print("\n=== Nuevas características temporales ===")
    temporal_cols = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'period_of_day', 'is_weekend']
    display(df_features[['date', 'hour'] + temporal_cols].head())

### 3.2 Características de Eventos

In [ ]:
# Generar características de eventos
if df is not None:
    df_features = feature_eng.create_event_features(df_features)
    
    print("\n=== Características de eventos ===")
    event_cols = ['events_in_window', 'time_since_last', 'time_to_next', 'events_per_hour']
    display(df_features[['date'] + event_cols].head(10))

### 3.3 Características de Texto

In [ ]:
# Generar características de texto
if df is not None:
    df_features = feature_eng.create_text_features(df_features)
    
    print("\n=== Características de texto ===")
    text_cols = ['text_length', 'word_count', 'has_alarma', 'has_movimiento', 'has_puerta']
    display(df_features[['text'] + text_cols].head())

### 3.4 Características Agregadas

In [ ]:
# Generar características agregadas
if df is not None:
    df_features = feature_eng.create_aggregated_features(df_features)
    
    print("\n=== Características agregadas ===")
    agg_cols = ['events_same_day', 'avg_events_this_hour', 'photos_same_day']
    display(df_features[['date', 'hour'] + agg_cols].head())

### 3.5 Guardar Features

In [ ]:
# Guardar DataFrame con todas las características
if df is not None:
    feature_eng.save_features(df_features, 'events_features.csv')
    print(f"\n✓ Total de características: {len(df_features.columns)}")
    print(f"✓ Guardado en: data/processed/events_features.csv")

## 4. Análisis Exploratorio de Datos (EDA)

Realizaremos un análisis visual completo de los datos.

In [ ]:
# Inicializar el analizador EDA
if df is not None:
    eda_analyzer = EDAAnalyzer(config_path='../configs/config.yaml')
    print("✓ Analizador EDA inicializado")

### 4.1 Estadísticas Descriptivas

In [ ]:
# Generar estadísticas descriptivas
if df is not None:
    stats = eda_analyzer.generate_summary_stats(df)
    
    print("\n=== Estadísticas Detalladas ===")
    for key, value in stats.items():
        if isinstance(value, dict):
            print(f"\n{key.upper()}:")
            for k, v in value.items():
                print(f"  {k}: {v}")
        elif not isinstance(value, dict):
            print(f"{key}: {value}")

### 4.2 Serie Temporal de Eventos

In [ ]:
# Visualización interactiva con Plotly
if df is not None:
    daily_events = df.groupby(df['date'].dt.date).size().reset_index()
    daily_events.columns = ['date', 'events']
    
    fig = px.line(
        daily_events,
        x='date',
        y='events',
        title='Serie Temporal de Eventos de Seguridad',
        labels={'date': 'Fecha', 'events': 'Número de Eventos'}
    )
    fig.update_traces(line_color='#2ecc71', line_width=2)
    fig.update_layout(hovermode='x unified')
    fig.show()

### 4.3 Distribución Horaria

In [ ]:
# Distribución por hora del día
if df is not None and 'hour' in df.columns:
    hourly_dist = df['hour'].value_counts().sort_index()
    
    fig = px.bar(
        x=hourly_dist.index,
        y=hourly_dist.values,
        title='Distribución de Eventos por Hora del Día',
        labels={'x': 'Hora del Día', 'y': 'Número de Eventos'}
    )
    fig.update_traces(marker_color='#3498db')
    fig.show()

### 4.4 Heatmap de Actividad

In [ ]:
# Heatmap de eventos por hora y día de la semana
if df is not None and 'hour' in df.columns and 'day_name' in df.columns:
    pivot = df.groupby(['hour', 'day_name']).size().unstack(fill_value=0)
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot.reindex(columns=day_order, fill_value=0)
    
    fig = px.imshow(
        pivot,
        labels=dict(x='Día de la Semana', y='Hora del Día', color='Eventos'),
        title='Heatmap de Actividad: Eventos por Hora y Día',
        aspect='auto',
        color_continuous_scale='YlOrRd'
    )
    fig.show()

### 4.5 Análisis de Fotos

In [ ]:
# Análisis de eventos con fotografías
if df is not None and 'has_photo' in df.columns:
    photo_stats = df['has_photo'].value_counts()
    
    fig = go.Figure(data=[go.Pie(
        labels=['Sin Foto', 'Con Foto'],
        values=photo_stats.values,
        hole=.3,
        marker_colors=['#e74c3c', '#2ecc71']
    )])
    
    fig.update_layout(
        title='Proporción de Eventos con Fotografía',
        annotations=[dict(text='Fotos', x=0.5, y=0.5, font_size=20, showarrow=False)]
    )
    fig.show()
    
    print(f"\n✓ Eventos con foto: {photo_stats.get(True, 0)} ({photo_stats.get(True, 0)/len(df)*100:.1f}%)")
    print(f"✓ Eventos sin foto: {photo_stats.get(False, 0)} ({photo_stats.get(False, 0)/len(df)*100:.1f}%)")

### 4.6 Distribución por Día de la Semana

In [ ]:
# Eventos por día de la semana
if df is not None and 'day_name' in df.columns:
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = df['day_name'].value_counts().reindex(day_order)
    
    colors = ['#e74c3c' if day in ['Saturday', 'Sunday'] else '#3498db' for day in day_order]
    
    fig = go.Figure(data=[go.Bar(
        x=day_counts.index,
        y=day_counts.values,
        marker_color=colors
    )])
    
    fig.update_layout(
        title='Distribución de Eventos por Día de la Semana',
        xaxis_title='Día',
        yaxis_title='Número de Eventos'
    )
    fig.show()

### 4.7 Análisis de Periodos del Día

In [ ]:
# Análisis por periodo del día
if 'df_features' in locals() and 'period_of_day' in df_features.columns:
    period_counts = df_features['period_of_day'].value_counts()
    
    fig = px.pie(
        values=period_counts.values,
        names=period_counts.index,
        title='Distribución de Eventos por Periodo del Día'
    )
    fig.show()

## 5. Generar Informe Completo

In [ ]:
# Generar informe EDA completo (figuras estáticas + markdown)
if df is not None:
    eda_analyzer.plot_time_series(df)
    eda_analyzer.plot_hourly_distribution(df)
    eda_analyzer.plot_daily_distribution(df)
    eda_analyzer.plot_photo_analysis(df)
    eda_analyzer.generate_report(df, stats)
    
    print("\n" + "="*60)
    print("✓ Informe EDA generado exitosamente")
    print("  Ubicación: reports/informe_EDA.md")
    print("  Figuras: reports/figures/")
    print("="*60)

## 6. Conclusiones

In [ ]:
# Resumen final del análisis
if df is not None:
    print("=" * 70)
    print("RESUMEN DEL ANÁLISIS EDA Y FEATURE ENGINEERING")
    print("=" * 70)
    print(f"\n✓ Total de eventos analizados: {len(df):,}")
    print(f"✓ Características generadas: {len(df_features.columns) if 'df_features' in locals() else 'N/A'}")
    print(f"✓ Periodo analizado: {(df['date'].max() - df['date'].min()).days} días")
    print(f"✓ Promedio de eventos/día: {len(df) / ((df['date'].max() - df['date'].min()).days + 1):.2f}")
    
    if 'hour' in df.columns:
        peak_hour = df['hour'].mode()[0]
        print(f"✓ Hora pico de eventos: {peak_hour}:00")
    
    if 'day_name' in df.columns:
        peak_day = df['day_name'].mode()[0]
        print(f"✓ Día con más eventos: {peak_day}")
    
    print("\n📊 Archivos generados:")
    print("   - data/processed/events_features.csv")
    print("   - reports/informe_EDA.md")
    print("   - reports/figures/*.png")
    
    print("\n🎯 Próximos pasos sugeridos:")
    print("   1. Configurar Arduino con los sensores")
    print("   2. Implementar detección de objetos con OpenCV")
    print("   3. Crear sistema de alertas en tiempo real")
    print("   4. Entrenar modelos de ML para clasificación de eventos")
    
    print("\n" + "="*70)
    print("✓ ANÁLISIS COMPLETADO EXITOSAMENTE")
    print("="*70)